Parts Copyright (c) Modular Inc extracted from
    https://github.com/modularml/mojo/blob/main/examples/matmul.mojo

for fair and appropriate use here in a Jupyter notebook

Used the Modular Mojo (Mojo) kernel current on 15th Apr 2024 for test results discussed in the ReadMe

The Mojo benchmark is intended as a learning tool so the Matrix type is defined from basics, rather than built-in.

In [1]:
alias type = DType.float32

struct Matrix[rows: Int, cols: Int]:
    var data: DTypePointer[type]

    # Initialize zeroeing all values
    fn __init__(inout self):
        self.data = DTypePointer[type].alloc(rows * cols)
        memset_zero(self.data, rows * cols)

    # Initialize taking a pointer, don't set any elements
    fn __init__(inout self, data: DTypePointer[type]):
        self.data = data

    # Initialize with random values
    @staticmethod
    fn rand() -> Self:
        var data = DTypePointer[type].alloc(rows * cols)
        random.rand(data, rows * cols)
        return Self(data)

    fn __getitem__(self, y: Int, x: Int) -> Scalar[type]:
        return self.load[1](y, x)

    fn __setitem__(self, y: Int, x: Int, val: Scalar[type]):
        self.store[1](y, x, val)

    fn load[nelts: Int](self, y: Int, x: Int) -> SIMD[type, nelts]:
        return self.data.load[width=nelts](y * self.cols + x)

    fn store[nelts: Int](self, y: Int, x: Int, val: SIMD[type, nelts]):
        return self.data.store[width=nelts](y * self.cols + x, val)

The Naive version allows the A and B looks like it should be slow but the optimiser will end up making it x faster than the CmnDot approach.  This probably reflects some optimization using AVX that is recognized for the naive case.

In [2]:
# Note that C, A, and B have types.
fn matmul_naive(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            for n in range(C.cols):
                C[m, n] += A[m, k] * B[k, n]

fn matmul_CmnDot(C: Matrix, A: Matrix, B: Matrix):
    var Cmn: Float32 = 0
    for m in range(C.rows):
        for n in range(C.cols):
            Cmn = 0
            for k in range(A.cols):
                Cmn += A[m, k] * B[k, n]
            C[m,n] = Cmn

The Mojo bench creates and frees the C, A, B arrays every trial.
This reflects the Mojo philosophy of tightly designed minimal data lifetime.

In [3]:
alias M = 1024
alias N = 1024
alias K = 1024

import benchmark

@always_inline
fn bench[
    func: fn (Matrix, Matrix, Matrix) -> None]():
    var C = Matrix[M, N]()
    var A = Matrix[M, K].rand()
    var B = Matrix[K, N].rand()

    @always_inline
    @parameter
    fn test_fn():
        _ = func(C, A, B)

    var secs = benchmark.run[test_fn](max_runtime_secs=1).mean()

    print (C[0,0], ", ", C[1,0], ", ", C[0,1000], ", ", C[1000,0])
    A.data.free()
    B.data.free()
    C.data.free()

# it is conventional to count "+=" as 2 FLOPs even though the fused multiply-add is pipelined as one op.

    var gflops = ((2 * M * N * K) / secs) / 1e9

    print(gflops, "GFLOP/s")

In [4]:
bench[matmul_naive]()

1250.37939453125 ,  1230.5911865234375 ,  1314.1461181640625 ,  1216.9930419921875
4.2374179621743062 GFLOP/s


The dot product in the inner loop, is slower than naive.
Julia runs at identical speed probably making the same optimization decision.

In [5]:
bench[matmul_CmnDot]()

251.63015747070312 ,  254.29837036132812 ,  249.54191589355469 ,  255.52023315429688
1.5286133724283093 GFLOP/s


No surprise that Mojo has us rewrite the function to vectorize it.

In [6]:
# Simplify the code by using the builtin vectorize function
from algorithm import vectorize

alias nelts = simdwidthof[type]() * 2

fn matmul_vectorized_1(C: Matrix, A: Matrix, B: Matrix):
    for m in range(C.rows):
        for k in range(A.cols):
            @parameter
            fn dot[nelts: Int](n: Int):
                C.store(m, n, C.load[nelts](m, n) + A[m, k] * B.load[nelts](k, n))
            vectorize[dot, nelts, size = C.cols]()

On my 8 core Intel Gen 11 this goes 6x faster than the naive version, around 27 GFLOPs.

In [7]:
bench[matmul_vectorized_1]()

9920.544921875 ,  10347.28125 ,  10458.8271484375 ,  10580.2578125
27.139883246045727 GFLOP/s


Parallelizing is not a large change to the code, but it converts the outer For loop to a function call for one row and then iterates the rows in parallel.

My 8 core Intel Gen 11 ends up a little more than 4x faster when parallel, at around 116 GFLOPs.

In [8]:
# Parallelize the code by using the builtin parallelize function
from algorithm import parallelize

fn matmul_parallelized(C: Matrix, A: Matrix, B: Matrix):
    @parameter
    fn calc_row(m: Int):
        for k in range(A.cols):
            @parameter
            fn dot[nelts : Int](n : Int):
                C.store[nelts](m,n, C.load[nelts](m,n) + A[m,k] * B.load[nelts](k,n))
            vectorize[dot, nelts, size = C.cols]()
    parallelize[calc_row](C.rows, C.rows)

In [9]:
bench[matmul_parallelized]()

25836.423828125 ,  25676.44921875 ,  27097.4375 ,  26004.072265625
116.38407916540572 GFLOP/s
